<a href="https://colab.research.google.com/github/JSJeong-me/GPT_Ochestration/blob/main/Deterministic_outputs_with_the_seed_parameter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to make your completions outputs consistent with the new seed parameter

**TLDR**: Developers can now specify `seed` parameter in the Chat Completion request for consistent completions. We always include a `system_fingerprint` in the response that helps developers understand changes in our system that will affect determinism.

### Context

Determinism has always been a big request from user communities when using our APIs. For instance, when granted the capability of getting deterministic numerical result, users can unlock quite a bit of use cases that’s sensitive to numerical changes.

#### Model level features for consistent outputs

The Chat Completions and Completions APIs are non-deterministic by default (which means model outputs may differ from request to request), but now offer some control towards deterministic outputs using a few model level controls.

This can unlock consistent completions which enables full control on the model behaviors for anything built on top of the APIs, and quite useful for reproducing results and testing so you know get peace of mind from knowing exactly what you’d get.

#### Implementing consistent outputs

To receive _mostly_ deterministic outputs across API calls:

- Set the `seed` parameter to any integer of your choice, but use the same value across requests. For example, `12345`.
- Set all other parameters (prompt, temperature, top_p, etc.) to the same values across requests.
- In the response, check the `system_fingerprint` field. The system fingerprint is an identifier for the current combination of model weights, infrastructure, and other configuration options used by OpenAI servers to generate the completion. It changes whenever you change request parameters, or OpenAI updates numerical configuration of the infrastructure serving our models (which may happen a few times a year).

If the `seed`, request parameters, and `system_fingerprint` all match across your requests, then model outputs will mostly be identical. There is a small chance that responses differ even when request parameters and `system_fingerprint` match, due to the inherent non-determinism of computers.


### Model level controls for consistent outputs - `seed` and `system_fingerprint`

##### `seed`

If specified, our system will make a best effort to sample deterministically, such that repeated requests with the same seed and parameters should return the same result. Determinism is not guaranteed, and you should refer to the `system_fingerprint` response parameter to monitor changes in the backend.

##### `system_fingerprint`

This fingerprint represents the backend configuration that the model runs with. It can be used in conjunction with the seed request parameter to understand when backend changes have been made that might impact determinism.This is the indicator on whether users should expect "almost always the same result".


## Example: Generating a consistent short story with a fixed seed

In this example, we will demonstrate how to generate a consistent short story using a fixed seed. This can be particularly useful in scenarios where you need to reproduce the same results for testing, debugging, or for applications that require consistent outputs.


In [2]:
!pip install scipy
!pip install tenacity
!pip install tiktoken
!pip install termcolor
!pip install openai
!pip install requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.8/217.8 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [3]:
import asyncio
import openai
import pprint
import difflib
from IPython.display import display, HTML

GPT_MODEL = "gpt-3.5-turbo-1106"

In [4]:
# Set up the OpenAI API client
openai.api_key = "sk-"

In [5]:
async def get_chat_response(system_message: str, user_request: str, seed: int = None):
    try:
        messages = [
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_request},
        ]

        response = openai.ChatCompletion.create(
            model=GPT_MODEL,
            messages=messages,
            seed=seed,
            max_tokens=200,
            temperature=0.7,
        )

        response_content = response["choices"][0]["message"]["content"]
        system_fingerprint = response["system_fingerprint"]
        prompt_tokens = response["usage"]["prompt_tokens"]
        completion_tokens = (
            response["usage"]["total_tokens"] - response["usage"]["prompt_tokens"]
        )

        table = f"""
        <table>
        <tr><th>Response</th><td>{response_content}</td></tr>
        <tr><th>System Fingerprint</th><td>{system_fingerprint}</td></tr>
        <tr><th>Number of prompt tokens</th><td>{prompt_tokens}</td></tr>
        <tr><th>Number of completion tokens</th><td>{completion_tokens}</td></tr>
        </table>
        """
        display(HTML(table))

        return response_content
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


# This function compares two responses and displays the differences in a table.
# Deletions are highlighted in red and additions are highlighted in green.
# If no differences are found, it prints "No differences found."


def compare_responses(previous_response: str, response: str):
    d = difflib.Differ()
    diff = d.compare(previous_response.splitlines(), response.splitlines())

    diff_table = "<table>"
    diff_exists = False

    for line in diff:
        if line.startswith("- "):
            diff_table += f"<tr style='color: red;'><td>{line}</td></tr>"
            diff_exists = True
        elif line.startswith("+ "):
            diff_table += f"<tr style='color: green;'><td>{line}</td></tr>"
            diff_exists = True
        else:
            diff_table += f"<tr><td>{line}</td></tr>"

    diff_table += "</table>"

    if diff_exists:
        display(HTML(diff_table))
    else:
        print("No differences found.")

First, let's try generating a short story about "a journey to Mars" without the `seed` parameter. This is the default behavior:


In [6]:
topic = "a journey to Mars"
system_message = "You are a helpful assistant that generates short stories."
user_request = f"Generate a short story about {topic}."

previous_response = await get_chat_response(
    system_message=system_message, user_request=user_request
)

response = await get_chat_response(
    system_message=system_message, user_request=user_request
)

# The function compare_responses is then called with the two responses as arguments.
# This function will compare the two responses and display the differences in a table.
# If no differences are found, it will print "No differences found."
compare_responses(previous_response, response)

An error occurred: module 'openai' has no attribute 'ChatCompletion'
An error occurred: module 'openai' has no attribute 'ChatCompletion'


AttributeError: ignored

Now, let's try to generate the short story with the same topic (a journey to Mars) with a constant `seed` of 123 and compare the responses and `system_fingerprint`.


In [7]:
SEED = 123
response = await get_chat_response(
    system_message=system_message, seed=SEED, user_request=user_request
)
previous_response = response
response = await get_chat_response(
    system_message=system_message, seed=SEED, user_request=user_request
)

compare_responses(previous_response, response)

An error occurred: module 'openai' has no attribute 'ChatCompletion'
An error occurred: module 'openai' has no attribute 'ChatCompletion'


AttributeError: ignored

## Conclusion

We demonstrated how to use a fixed integer `seed` to generate consistent outputs from our model.This is particularly useful in scenarios where reproducibility is important. However, it's important to note that while the `seed` ensures consistency, it does not guarantee the quality of the output. For instance, in the example provided, we used the same seed to generate a short story about a journey to Mars. Despite querying the model multiple times, the output remained consistent, demonstrating the effectiveness of using this model level control for reproducibility. Another great extension of this could be to use consistent `seed` when benchmarking/evaluating the performance of different prompts or models, to ensure that each version is evaluated under the same conditions, making the comparisons fair and the results reliable.


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=baca1bec-71a4-4a27-a9d1-210d39960b44' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>
